# Create redo records
This Jupyter notebook shows how to create a Senzing "redo record". It assums a G2 database that is empty

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Engine;
import com.senzing.g2.engine.G2JNI;
import com.senzing.g2.engine.G2ConfigMgr;
import com.senzing.g2.engine.G2ConfigMgrJNI;
import com.senzing.g2.engine.G2Config;
import com.senzing.g2.engine.G2ConfigJNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
public static void RenderJSON(Object obj){
    String str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.
String senzingConfigJson=System.getenv("SENZING_ENGINE_CONFIGURATION_JSON");

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

    // Construct the JSON string used for Senzing Engine configuration.
if(senzingConfigJson==null)
    senzingConfigJson = "{"
       + "\"PIPELINE\": {"
       +     "\"CONFIGPATH\": \"" + configPath + "\","
       +     "\"SUPPORTPATH\": \"" + supportPath + "\","
       +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
       + "},"
       + "\"SQL\": {"
       +     "\"CONNECTION\": \"" + sqlConnection + "\""
       + "}}";

RenderJSON(senzingConfigJson);

## G2Engine

### Senzing initializaton
Create an instance of G2Engine, Config, and ConfigMgr

In [ ]:
G2Engine g2engine = new G2JNI();

String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.init(moduleName, senzingConfigJson, verboseLogging);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

In [ ]:
String moduleName = "ExampleG2Config";
boolean verboseLogging = true;
Result<Long> config = new Result<Long>();
G2Config g2config = new G2ConfigJNI();
int return_code = g2config.init(moduleName, senzingConfigJson, verboseLogging);
g2config.create(config);
long config_handle = config.getValue();
if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

In [ ]:
G2ConfigMgr g2ConfigMgr = new G2ConfigMgrJNI();

String moduleName = "ExampleG2ConfigMgr";
boolean verboseLogging = true;
int returnCode = g2ConfigMgr.init(moduleName, senzingConfigJson, verboseLogging);
g2ConfigMgr.getLastException();
System.out.print(returnCode);

### Create add data source function

In [ ]:
public void addDataSource(int dataSource_suff){
    StringBuffer response = new StringBuffer();
    String dataSource_prefix = "TEST_DATA_SOURCE_";
    String dataSourceID = dataSource_prefix + dataSource_suff;
    String config_comment = "Added "+dataSourceID;
    
    g2config.addDataSource(config_handle, dataSourceID, response);
    
    StringBuffer config_buffer = new StringBuffer();
    
    g2config.save(config_handle, config_buffer);
    
    String config_json = config_buffer.toString();
    Result<Long> configID = new Result<Long>();
    
    g2ConfigMgr.addConfig(config_json, config_comment, configID);
    g2engine.reinit(Long.parseLong(configID.getValue().toString()));
}

In [ ]:
String loadId = null;
public void addRecord(int recordid_suff, int datasource_suff){
    String dataSourcePre = "TEST_DATA_SOURCE_";
    String recordIdPre = "RECORD_";
    String dataSourceId = dataSourcePre + datasource_suff;
    String recordId = recordIdPre + recordid_suff;
    String data = "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_LAST\": \"Smith\",\"NAME_FIRST\": \"John\", \"NAME_MIDDLE\": \"M\", \"PASSPORT_NUMBER\": \"PP11111\", \"PASSPORT_COUNTRY\": \"US\", \"DRIVERS_LICENSE_NUMBER\": \"DL1" + (000+recordid_suff) + "\", \"SSN_NUMBER\": \"111-11-1111\"}";
    int return_code  = g2engine.addRecord(dataSourceId, recordId, data, loadId);
    if(return_code!=0)
        System.out.print(g2engine.getLastException());
}

In [ ]:
addDataSource(1);
addRecord(1, 1);
addRecord(2, 1);
addDataSource(2);
addRecord(3, 2);
addRecord(4, 2);
addDataSource(3);
addRecord(5, 3);
addRecord(6, 3);

## Delete record
Deleting a record will create a "redo record".

In [ ]:
g2engine.deleteRecord("TEST_DATA_SOURCE_3", "RECORD_5", loadId);

In [ ]:
long redoRecords = g2engine.countRedoRecords();
System.out.print("Number of redo records: "+redoRecords);

### Print data sources
Print the list of currently defined data sources

In [ ]:
StringBuffer datasources = new StringBuffer();

int return_code = g2config.listDataSources(config_handle, datasources);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(datasources);